In [1]:
#Importing Libraries
from PIL import Image, ImageDraw
import itertools
import numpy as np

In [2]:
#PREPROCESSING
img = Image.open("sign.jpg")
img= img.convert('L') # convert image to 8 bit black and white
image_file = Image.new( img.mode, img.size)
pixelsNew = image_file.load()
for x,y in itertools.product(range(0,img.size[0]),range(0,img.size[1])):
    color = img.getpixel((x,y))
    if color>=125:
        pixelsNew[(x, y)]= 255
    else:
        pixelsNew[(x, y)] = 0
#Now image_file is converted to 1 bit black and white image

In [3]:
#defining bounding box for the image
def coordinates(image_file):
    left,top= image_file.size#widthxheight
    right,bottom = 0,0
    for x,y in itertools.product(range(0,image_file.size[0]),range(0,image_file.size[1])):
        color = image_file.getpixel((x,y))
        if color is 0:
            if x>right:
                right = x
            if x<left:
                left =x
            if y>bottom:
                bottom = y
            if y<top:
                top = y
    return left,right,top,bottom

In [4]:
#TASK 2
#FInding centroid of the image defined by the dimentions
def centroid(image_file,left,right,top,bottom):
    width =abs(left-right)
    height= abs(top-bottom)
    cx = left
    cy = top
    n=1  #as we don't divide by 0 
    for x,y in itertools.product(range(width),range(height)):
        try:
            if image_file.getpixel((x+left,y+top)) is 0:
                cx+=left+x
                cy+=top+y
                n+=1
        except:
            print("Image index out of bound")
    if(n>1):
        n -= 1  #for accuracy
    cx=cx//n
    cy=cy//n
    return cx,cy

In [5]:
#white transaction
def whitetrans(image_file,left,right,top,bottom):
    width=abs(right-left)
    height=abs(bottom-top)
    prev = image_file.getpixel((0,0))
    n=0
    for x,y in itertools.product(range(left+1,right,-1),range(top+1,bottom-1)):
        cur = image_file.getpixel((x,y))
        if cur is 255 and prev is 0:
            n=n+1
        prev = cur
    return n

In [6]:
def findRatio(left,right,top,bottom):
    width=abs(right-left)
    height=abs(bottom-top)
    if height == 0:
        return 0
    return width/height

In [7]:
#TASK 3
t=[]
r=[]
centr =[]
def split(image_file,left, right, top, bottom, depth=0):
    cx,cy=centroid(image_file,left,right,top,bottom)
    if depth < 3:
        split(image_file, left, cx, top, cy, depth + 1)
        split(image_file, cx, right, top, cy, depth + 1)
        split(image_file, left, cx, cy, bottom, depth + 1)
        split(image_file, cx, right, cy, bottom, depth + 1)
    else:
        t.append(whitetrans(image_file, left, right, top, bottom))
        r.append(findRatio(left,right,top,bottom))
        centr.append(str(cx)+" "+str(cy))
    draw = ImageDraw.Draw(image_file)
    draw.rectangle(((left, top), (right, bottom)))#drawing lines on image

In [8]:
left,right,top,bottom = coordinates(image_file) 
split(image_file,left,right,top,bottom)
#saving them in files
np.savetxt("transaction.txt", t, delimiter='\n',fmt='%d')
np.savetxt("Aspectratio.txt", r, delimiter='\n',fmt='%d')
np.savetxt("Center.txt", centr, delimiter='\n',fmt='%s')
image_file.show()

In [9]:
#END